# Library

In [ ]:
!pip install langchain_chroma faiss-cpu lark

In [1]:
from src.utils import show_stream, naver_news_crawler

from dotenv import load_dotenv

import os
import numpy as np
import pickle
import json
import bs4
from bs4 import BeautifulSoup
import requests
import uuid
import nest_asyncio
from tqdm import tqdm
from operator import itemgetter
from datetime import datetime
from pydantic import BaseModel, Field
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from seaborn import load_dataset

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_core.load import dumpd, dumps, load, loads
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.callbacks.manager import get_openai_callback
from langchain_community.chat_models import ChatOllama
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
    MaxMarginalRelevanceExampleSelector,
)
from langchain_core.output_parsers import (
    StrOutputParser,
    PydanticOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser,
)
from langchain.output_parsers.structured import (
    ResponseSchema,
    StructuredOutputParser,
)
from langchain.output_parsers.pandas_dataframe import PandasDataFrameOutputParser
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from langchain.output_parsers.retry import RetryOutputParser
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda,
    ConfigurableField,
)

from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import (
    UnstructuredPDFLoader,
    PyPDFium2Loader,
    PDFMinerLoader,
    PyPDFDirectoryLoader,
    PDFPlumberLoader,
    UnstructuredExcelLoader,
    DataFrameLoader,
    Docx2txtLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredPowerPointLoader,
    WebBaseLoader,
    TextLoader,
    DirectoryLoader,
    JSONLoader,
    ArxivLoader,
)

from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    Language,
    MarkdownHeaderTextSplitter,
    HTMLHeaderTextSplitter,
    RecursiveJsonSplitter,
)
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.storage import (
    LocalFileStore,
    InMemoryByteStore,
    InMemoryStore,
)
from langchain.embeddings import CacheBackedEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_chroma import Chroma

from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import (
    ContextualCompressionRetriever,
    BM25Retriever,
    EnsembleRetriever,
    ParentDocumentRetriever,
    TimeWeightedVectorStoreRetriever,
)
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\ProgramData\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
load_dotenv()
set_llm_cache(InMemoryCache())

# LangChain

자연어 처리와 대화형 AI 애플리케이션을 구축하는 데 도움을 주는 프레임워크. <br>
여러 가지 NLP 모델과 데이터 소스를 쉽게 통합할 수 있도록 설계. <br>
복잡한 대화형 시스템을 개발하는 데 필요한 다양한 도구와 컴포넌트를 제공. <br>
대화형 애플리케이션, 챗봇, 자동화된 고객 서비스 솔루션 등을 보다 효율적으로 개발 가능. <br>

<br>

<font style="font-size:20px"> 특징 </font>

- 모델 통합: 다양한 언어 모델(예: GPT, Claude 등)을 쉽게 사용할 수 있도록 지원
- 데이터 소스 연결: 데이터베이스, API, 파일 시스템 등 여러 소스와 연결할 수 있는 기능 제공
- 체인 구성: 여러 작업을 순차적으로 연결하여 복잡한 프로세스를 만들 수 있는 체인 개념 도입
- 사용자 정의: 사용자 요구에 맞게 쉽게 확장하고 커스터마이즈 가능

## RAG

Retrieval-Augmented Generation(RAG)는 정보 검색과 생성을 통합하는 방법론. <br>
RAG는 대규모 문서 데이터베이스에서 관련 정보를 검색하고, 이를 통해 모델이 더 정확하고 상세한 답변을 생성할 수 있도록 함. <br>
최신 뉴스 이벤트나 특정 분야의 전문 지식과 같은 주제에 대해 물어보면, RAG는 관련 문서를 찾아 그 내용을 바탕으로 답변.

|사전 준비 단계|Runtime 단계|
|-----------|-----------|
|![](https://wikidocs.net/images/page/233780/rag-graphic-1.png)|![](https://wikidocs.net/images/page/233780/rag-graphic-2.png)|
|Document Load: 외부 데이터 소스에서 필요한 문서를 로드|Retriever: 질문이 주어지면 관련된 내용을 벡터 데이터베이스에서 검색|
|Text Splitter: 로드된 문서를 처리 가능한 작은 단위로 분할|Prompt: 검색된 정보를 바탕으로 언어 모델을 위한 질문 구성|
|Embedding: 각 문서 또는 문서의 일부를 벡터 형태로 변환|LLM: 구성된 프롬프트를 사용하여 언어 모델이 답변 생성|
|Vector Store: 임베딩된 벡터들을 데이터베이스에 저장. 요약된 키워드를 색인화하여 나중에 빠르게 찾을 수 있도록 함|Chain: 이전의 모든 과정의 하나의 파이프라인으로 묶어주는 Chain 생성|

### Embedding

한국어 검색을 위한 임베딩 벤치마크: https://github.com/teddylee777/Kor-IR?tab=readme-ov-file

#### OpenAIEmbeddings

OpenAI의 embeding API를 사용한 embedding.

<br>

> ```python
> embedding_model = OpenAIEmbeddings(
>     model='text-embedding-3-small',
>     # dimensions=1024, # 차원 조정
> )
> 
> # text embedding
> embedded_text = embedding_model.embed_query(text)
> 
> # document embedding
> embedded_documents = embeddings.embed_documents([text])
> ```

In [102]:
embedding_model = OpenAIEmbeddings(
    model='text-embedding-3-small'
)

In [117]:
# 'hello world!'라는 쿼리를 임베딩하여 numpy 배열로 변환
a = np.array(embedding_model.embed_query('hello world!'))

# 'hello world'라는 쿼리를 임베딩하여 numpy 배열로 변환
b = np.array(embedding_model.embed_query('hello world'))

# 코사인 유사도 계산
# a와 b의 내적을 a의 노름과 b의 노름으로 나누어 코사인 유사도를 구함
cos_sim = (a @ b.T) / (np.linalg.norm(a) * np.linalg.norm(b))

# 계산된 코사인 유사도 출력
cos_sim

0.9341421221189211

In [119]:
# 여러 문서에 대해 임베딩을 생성
# 'hello world!'와 'hello world'라는 두 개의 문서가 포함된 리스트를 전달
np.array(embedding_model.embed_documents([
    'hello world!',
    'hello world',
])).shape    # 생성된 임베딩의 형태(shape)를 확인

(2, 1536)

In [131]:
# nlp_keywords를 text loader를 통해서 load
loader = TextLoader('./data/nlp_keywords.txt')
document = loader.load()

# splitter를 통해서 chunk 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
)
documents = text_splitter.create_documents([document[0].page_content])

# 분할된 데이터를 embedding
documents = [document.page_content for document in documents]
model_embedding = OpenAIEmbeddings(model='text-embedding-3-small')
documents_embedded = model_embedding.embed_documents(documents)
np.array(documents_embedded).shape

(20, 1536)

In [138]:
cos_sim = cosine_similarity(np.array(documents_embedded))
np.fill_diagonal(cos_sim, 0)
threshold = 0.7
np.where(cos_sim > threshold)

(array([14, 15], dtype=int64), array([15, 14], dtype=int64))

In [139]:
np.array(documents)[[14, 15]]

array(['Token\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nTokenizer',
       'Tokenizer\n\n정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nVectorStore'],
      dtype='<U215')

#### CacheBackedEmbeddings

재계산을 피하기 위해 저장하거나 일시적으로 caching. <br>
embeddings을 key-value 저장소에 caching하는 wrapper. <br>
텍스트는 해시되고 이 해시는 캐시에서 키로 사용. <br>

<br>

CacheBackedEmbeddings:
- underlying_embeddings: embedding을 위해 사용하는 모델.
- document_embedding_cache: 문서 임베딩을 캐싱하기 위한 ByteStore.
- namespace (optional): 다른 캐시와의 충돌을 피하기 위해 사용하는 네임스페이스.

<br>

> ```python
> store = LocalFileStore("./cache/")    # 영구적
> store = InMemoryByteStore()           # 비영구적
> 
> cached_embedder = CacheBackedEmbeddings.from_bytes_store(
>     underlying_embeddings=embedding,
>     document_embedding_cache=store,
>     namespace=embedding.model,
> )
> ```

In [142]:
model_embedding = OpenAIEmbeddings(
    model='text-embedding-3-small'
)

store = LocalFileStore('./cache/')   # 비휘발성 (하드)
store = InmemoryByteStore()          # 휘발성   (메모리)

model_cache_embedding = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=model_embedding,
    document_embedding_cache=store,
    namespace=model_embedding.model,
)

#### HuggingFace Embeddings

huggingface endpoint를 통한 embedding. <br>

<br>

> ```python
> embedding_model = HuggingFaceEndpointEmbeddings(
>     model='BAAI/bge-m3',
>     task='feature-extraction',
>     # model_kwargs: optional ={'device': 'cuda'},  # cuda, cpu
>     # encode_kwargs={'normalize_embeddings': True},
> )
> 
> # 문서 임베딩
> embedded_documents = embedding_model.embed_documents(texts)
> ```

In [149]:
model_embedding = HuggingFaceEndpointEmbeddings(
    model='BAAI/bge-m3',
    task='feature-extraction',
)

model_embedding.embed_documents(['hello world'])

[[-0.04039410501718521,
  0.0370350144803524,
  -0.028974439948797226,
  0.01611720770597458,
  -0.03569155931472778,
  -0.04073759913444519,
  -0.0550544299185276,
  -0.040867533534765244,
  0.003238329663872719,
  0.001956742024049163,
  -0.006565415300428867,
  0.011655457317829132,
  0.021531231701374054,
  -0.013461880385875702,
  0.030359994620084763,
  0.0038853585720062256,
  0.02051585167646408,
  -0.02315090410411358,
  -0.021936513483524323,
  -0.03128373250365257,
  -0.05100717768073082,
  -0.01739712432026863,
  0.0123360026627779,
  -0.014076601713895798,
  0.015406228601932526,
  0.04133284091949463,
  -0.036864809691905975,
  -0.0005830508889630437,
  0.00033308894489891827,
  -0.06086040660738945,
  0.037107087671756744,
  0.08012109249830246,
  -0.017788074910640717,
  -0.04983343556523323,
  -0.03419807553291321,
  -0.035880349576473236,
  0.004637434612959623,
  0.0005803846870549023,
  -0.06894704699516296,
  0.010874012485146523,
  0.01592210866510868,
  0.0225864

#### OllamaEmbeddings

ollama를 통한 embedding. <br>

<br>

> ```python
> ollama_embeddings = OllamaEmbeddings(
>     model=<model_name>,
> )
> 
> # 문서 임베딩
> embedded_documents = ollama_embeddings.embed_documents(texts)
> ```

In [ ]:
model_embedding = OllamaEmbeddings(
    model='llama3.2:1b'
)
model_embedding.embed_documents(['hello world'])

### Vector Database

[langchain 지원 db](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

[VectorDB 비교](https://discuss.pytorch.kr/t/2023-picking-a-vector-database-a-comparison-and-guide-for-2023/2625)

임베딩된 벡터들을 효율적으로 저장하고 관리하는 과정. <br>
이 단계는 향후 검색 과정에서 벡터들을 빠르게 조회하고, 관련 문서를 신속하게 찾아내는 데 중요. <br>

<br>

<font style="font-size:20px"> 특징 </font>

- 빠른 검색 속도: 임베딩 벡터들을 효과적으로 저장하고 색인화함으로써, 대량의 데이터 중에서도 관련된 정보를 빠르게 검색 가능. <br>
- Scailability: 데이터가 지속적으로 증가함에 따라 이를 수용할 수 있는 충분한 scailability를 제공해야 함. 효율적인 저장 구조는 데이터베이스의 확장성을 보장하며, 시스템의 성능 저하 없이 대규모 데이터를 관리할 수 있도록 함.
- Semantic Search: 사용자의 질문과 의미상으로 유사한 단락 조회.

#### FAISS

paper: [FAISS](https://arxiv.org/pdf/1702.08734) <br>
blog: [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)

reference: [FAISS](https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/)

documentation: [FAISS](https://faiss.ai/)

FAISS: Facebook AI Similarity Search <br>
밀집 벡터의 효율적인 유사도 검색과 클러스터링을 위한 library. <br>

##### Create

> ```python
> dimension_size = 1536
>
> # FAISS Vector Store 생성
> db = FAISS(
>     embedding_function=OpenAIEmbeddings(),
>     index=faiss.IndexFlatL2(dimension_size),
>     docstore=InMemoryDocstore(),
>     index_to_docstore_id={},
> )
>
> # Document로부터 FAISS Vector Store 생성
> db = FAISS.from_documents(
>     documents=documents,
>     embedding=OpenAIEmbeddings(),
> )
>
> # text로부터 FAISS Vector Store 생성
> db = FAISS.from_texts(
>     texts=texts,
>     embedding=OpenAIEmbeddings(),
>     metadatas=None,
>     ids=None,
> )
>
> db.index_to_docstore_id # 문서 저장소 ID 확인
> db.docstore._dict       # 저장된 문서의 ID: Document
> ```

In [181]:
loader = TextLoader('./data/nlp_keywords.txt')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
)
documents = loader.load_and_split(text_splitter)

db = FAISS.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
)

In [163]:
db.index_to_docstore_id

{0: 'f48b3541-b55f-4f67-b828-5175e5fd94ad',
 1: '8d113398-d5e8-40af-b5dc-3cdfce382f13',
 2: 'fdc229cf-1bc1-45e2-8603-f508e3ee5f9f',
 3: 'c3bef6e6-c1f6-4394-b25e-8f93e62d0dc9',
 4: 'db52abbb-2d71-45b9-8c56-7623f8a19a39',
 5: '096ac4ca-654a-4453-8d69-8ffb3160e530',
 6: '8704f47a-015d-46ac-9b85-a19d83f67d72',
 7: '1339500a-343b-4229-b9d3-9d23a68f5071',
 8: '726a8782-251a-4895-878b-c097ef229cd2',
 9: 'b5e4443b-df91-4c69-bc4e-7fa1a9cb2e48',
 10: '8e347d59-c599-44f8-8f89-a84ea02c9cde',
 11: 'f98312e2-6610-423a-904e-55c71e257658',
 12: 'f8b81d8a-44fc-44e3-bb46-ad910b322ec7',
 13: '3598b1ff-2bd8-453c-8a38-5c6aaab96f72',
 14: 'a00408ce-c968-40c9-a432-1747484fd362',
 15: '1658521e-7ebf-47cf-93b8-1fc7870c3977',
 16: '5a4ed1c9-71af-4369-b85a-60d0de4af9cb',
 17: '48fbd35d-c6f7-45f6-ae81-4784b0ea653d',
 18: '38719c94-791e-41ef-ad8a-2d1e2c1f98b3',
 19: 'd259c738-dc28-467d-9bd1-b9033b88db72'}

In [165]:
db.docstore._dict

{'f48b3541-b55f-4f67-b828-5175e5fd94ad': Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍스트 생성, 딥러닝\n\nAttention 메커니즘'),
 '8d113398-d5e8-40af-b5dc-3cdfce382f13': Document(metadata={'source': './data/nlp_keywords.txt'}, page_content="Attention 메커니즘\n\n정의: Attention 메커니즘은 딥러닝에서 중요한 정보에 더 많은 '주의'를 기울이도록 하는 기법입니다.\n예시: 번역 모델에서 Attention 메커니즘은 입력 문장의 중요한 부분에 더 집중하여 정확한 번역을 생성합니다.\n연관키워드: 딥러닝, 자연어 처리, 시퀀스 모델링\n\n판다스 (Pandas)"),
 'fdc229cf-1bc1-45e2-8603-f508e3ee5f9f': Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='판다스 (Pandas)\n\n정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조작 도구를 제공하는 라이브러리입니다.\n예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며, 다양한 분석을 수행할 수 있습니다.\n연관키워드: 데이터 분석, 파이썬, 데이터 처리\n\nDeep Learning'),
 'c3bef6e6-c1f6-4394-b25e-8f93e62d0dc9': Document(metadata={'source': './data/nlp

In [171]:
db_temp = FAISS.from_texts(
    ['hello', 'hi', 'nice to meet you'],
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
    metadatas=[
        {'source': 'greeting'},
        {'source': 'greeting'},
        {'source': 'greeting'},
    ],
    ids=['1', '2', '3']
)

In [172]:
db_temp.docstore._dict

{'1': Document(metadata={'source': 'greeting'}, page_content='hello'),
 '2': Document(metadata={'source': 'greeting'}, page_content='hi'),
 '3': Document(metadata={'source': 'greeting'}, page_content='nice to meet you')}

In [180]:
# data: SPRi AI brief
# pdf load
loader = PyPDFLoader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')
documents = loader.load()

# splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)
documents = text_splitter.split_documents(documents)

# to db
db = FAISS.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings(model='text-embedding-3-small')
)

##### Similarity Search

주어진 쿼리와 가장 유사한 문서 검색. <br>

<br>

> ```python
> db.similarity_search(
>     'TF IDF 에 대하여 알려줘',
>     # filter={'source': './data/keywords.txt'},
>     # fetch_k=20,   # 필터링 이전 가져올 문서 수.
>     # k=2,          # fetch_k로 가져온 문서 중에서 필터링을 거쳐 최종 몇 개의 문서를 선택할지 결정
> )
> ```

In [187]:
db.similarity_search(
    'GPT가 뭐야?',
    k=1,
    filter={
        'source': './data/nlp_keywords.txt'
    }
)

[Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍스트 생성, 딥러닝\n\nAttention 메커니즘')]

##### Add

DB에 문서를 추가하거나 업데이트. <br>

<br>

> ```python
> # Document 추가
> db.add_documents(
>     [
>         Document(
>             page_content='new document',
>           metadata={'source': 'my brain'},
>         )
>     ],
>     ids=['new_id1'],
> )
> 
> # text를 통한 추가
> db.add_texts(
>     ['new text data', 'new text data2'],
>     metadatas=[{'source': 'my brain'}, {'source': 'my brain'}],
>     ids=['new_id2', 'new_id3'],
> )
> ```

In [189]:
db.add_documents(
    [
        Document(
            page_content='test',
            metadata={'source': 'my_brain'}
        )
    ]
)

['cf6a667e-8d3b-47db-b21f-f1fc8e097a74']

In [190]:
db.similarity_search('test')

[Document(metadata={'source': 'my_brain'}, page_content='test'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='SQL\n\n정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다.\n예시: SELECT * FROM users WHERE age > 18;은 18세 이상의 사용자 정보를 조회합니다.\n연관키워드: 데이터베이스, 쿼리, 데이터 관리\n\nJSON'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='Transformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nSQL'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='Tokenizer\n\n정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nVectorStore')]

In [191]:
# 입력 텍스트: KOSPI, KOSDAQ
# metadata: 'source': 'KRX'

db.add_texts(
    ['KOSPI', 'KOSDAQ'],
    metadatas=[{'source': 'KRX'}, {'source': 'KRX'}],
)

['429a17ec-311a-4215-a953-8a6a43f67e8c',
 'ace820f5-d3c5-4bf5-b32b-d2117dda2bac']

In [193]:
db.similarity_search('KOS', k=2)

[Document(metadata={'source': 'KRX'}, page_content='KOSDAQ'),
 Document(metadata={'source': 'KRX'}, page_content='KOSPI')]

##### Delete

DB에서 특정 ID의 문서 제거 <br>

<br>

> ```python
> db.delete(ids)
> ```

In [195]:
db.delete(['cf6a667e-8d3b-47db-b21f-f1fc8e097a74'])

True

In [197]:
db.similarity_search('test')

[Document(metadata={'source': 'KRX'}, page_content='KOSDAQ'),
 Document(metadata={'source': 'KRX'}, page_content='KOSPI'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='SQL\n\n정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다.\n예시: SELECT * FROM users WHERE age > 18;은 18세 이상의 사용자 정보를 조회합니다.\n연관키워드: 데이터베이스, 쿼리, 데이터 관리\n\nJSON'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='Transformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nSQL')]

##### Save/Load

DB를 로컬 디스크에 저장. <br>

<br>

>```python
> # save
> db.save_local(
>     folder_path='faiss_db',
>     index_name='faiss_index',
> )
> 
> # load
> db = FAISS.load_local(
>     folder_path='faiss_db',
>     index_name='faiss_index',
>     embeddings=OpenAIEmbedding(),
>     allow_dangerous_deserialization=True,   # pickle 파일 역직렬화 허용 여부
> )
>```

In [202]:
db.save_local(
    folder_path='faiss_db',
    index_name='faiss_index',
)

In [205]:
db_temp = FAISS.load_local(
    folder_path='faiss_db',
    index_name='faiss_index',
    embeddings=OpenAIEmbeddings(model='text-embedding-3-small'),
    allow_dangerous_deserialization=True,
)

In [207]:
db_temp.similarity_search('KOS')

[Document(metadata={'source': 'KRX'}, page_content='KOSDAQ'),
 Document(metadata={'source': 'KRX'}, page_content='KOSPI'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='Transformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nSQL'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='JSON\n\n정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nCSV')]

##### Merge

두 FAISS 객체를 병합. <br>

<br>

> ```python
> db.merge_from(db2)
> ```

In [212]:
# data: finance_keywords
# load
loader = TextLoader('./data/finance_keywords.txt')

# splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
)
documents = loader.load_and_split(text_splitter)

# to db
db2 = FAISS.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings(model='text-embedding-3-small')
)

# merge
db.merge_from(db2)

In [216]:
db.similarity_search('헤지가 뭐야?')

[Document(metadata={'source': './data/finance_keywords.txt'}, page_content='헤지 (Hedge)\n\n정의: 헤지는 투자에서 손실을 줄이기 위해 위험을 줄이는 전략을 의미합니다.\n예시: 파생상품을 이용해 가격 변동 리스크를 헤지할 수 있습니다.\n연관키워드: 리스크 관리, 투자, 파생상품\n\n유가증권 (Securities)\n\n정의: 유가증권은 투자자에게 소유권이나 채권을 부여하는 재무 자산입니다.\n예시: 주식과 채권이 유가증권에 해당합니다.\n연관키워드: 투자, 금융 시장, 증권\n\n세금 (Tax)'),
 Document(metadata={'source': './data/finance_keywords.txt'}, page_content='시장 (Market)\n\n정의: 시장은 상품이나 서비스가 거래되는 공간을 의미합니다.\n예시: 주식 시장, 외환 시장 등이 있습니다.\n연관키워드: 경제, 거래, 투자\n\n재무제표 (Financial Statement)\n\n정의: 재무제표는 기업의 재무 상태와 성과를 나타내는 공식 문서입니다.\n예시: 손익계산서, 대차대조표가 재무제표의 일종입니다.\n연관키워드: 회계, 재무, 분석\n\n헤지 (Hedge)'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content="Attention 메커니즘\n\n정의: Attention 메커니즘은 딥러닝에서 중요한 정보에 더 많은 '주의'를 기울이도록 하는 기법입니다.\n예시: 번역 모델에서 Attention 메커니즘은 입력 문장의 중요한 부분에 더 집중하여 정확한 번역을 생성합니다.\n연관키워드: 딥러닝, 자연어 처리, 시퀀스 모델링\n\n판다스 (Pandas)"),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='Tokenizer\n\n정의: 토크나이저는 텍스트

##### as_retriever

현재 db를 기반으로 VectorStoreRetriver 객체 생성. <br>

MMR: $ \lambda \cdot \text{Sim}(d, Q) + (1 - \lambda) \cdot max_{d' \in D'} Sim (d, d') $
- $ \text{Sim}(d, Q) $: 문서와 쿼리 간 유사도
- $ max_{d' \in D'} Sim (d, d') $: 문서(d)와 이미 선택된 문서 집합 (D') 중 가장 유사한 문서와의 유사성
- $ \lambda $: 유사성과 다양성의 반영도를 조절하는 파라미터
    - $ \lambda = 1 $: 유사성만 고려
    - $ \lambda = 0 $: 다양성 최대화

<br>

> ```python
> # similarity
> retriever = db.as_retriever()
> retriever.invoke('tf-idf에 대해서 알려줘')
> 
> # mmr (maximal marginal relavance)
> retriever = db.as_retriever(
>     searcy_type='mmr',
>     search_kwargs={
>         'fetch_k': 20,        # mmr에 전달할 문서 수
>         'k': 6,               # 반환 무서 수
>         'lambda_mult': 0.25,  # 다양성 조정 파라미터
>         # 'filter': {'source': './data/keywords.txt'}        # 메타데이터 기반 필터링
>     }
> )
> retriever.invoke('tf-idf에 대해서 알려줘')
>
> # score threshold
> retriever = db.as_retriever(
>     search_type='similarity_score_threshold',
>     search_kwargs={'score_threshold': 0.8},
> )
> retriever.invoke('tf-idf에 대해서 알려줘') 
> ```

In [221]:
retriever = db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'fetch_k': 20,
        'k': 1,
        'lambda_mult': 0.3,
    }
)

In [222]:
retriever.invoke('GPT가 뭐야?')

[Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍스트 생성, 딥러닝\n\nAttention 메커니즘')]

#### Chroma

reference: [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)

<br>

개발자의 생산성과 행복에 초점을 맞춘 AI 네이티브 오픈 소스 벡터 데이터베이스. 

<br>

##### Create

> ```python
> # document로부터 Chroma Vector Store 생성
> db = Chroma.from_documents(
>     documents=documents,
>     embedding=OpenAIEmbeddings(),
>     collection_name='db',
>     # persist_directory=<db_path> # disk에 파일 형태로 저장
> )
> 
> # 텍스트로부터 Chroma Vector Store 생성
> db2 = Chroma.from_texts(
>     ['python', 'cpp'],
>     embedding=OpenAIEmbeddings(),
> )
> 
> db.get()    # 정보 확인
> ```

##### Similarity Search

주어진 쿼리와 가장 유사한 문서 검색. <br>

<br>

> ```python
> db.similarity_search(
>     'TF IDF 에 대하여 알려줘',
>     # filter={'source': './data/keywords.txt'},
>     # k=2,          # fetch_k로 가져온 문서 중에서 필터링을 거쳐 최종 몇 개의 문서를 선택할지 결정
> )
> ```

##### Add

DB에 문서를 추가하거나 업데이트. <br>

<br>

> ```python
> # Document 추가
> db.add_documents(
>     [
>         Document(
>             page_content='new document',
>             metadata={'source': 'my brain'},
>             id='1',    # 제공되지 않으면 자동생성
>         )
>     ],
> )
> 
> # text를 통한 추가
> db.add_texts(
>     ['new text data', 'new text data2'],
>     metadatas=[{'source': 'my brain'}, {'source': 'my brain'}],
>     ids=['new_id2', 'new_id3'],
> )
> ```

##### Delete

DB에서 특정 ID의 문서 제거 <br>

<br>

> ```python
> db.delete(ids)           # 특정 id 삭제
> db.reset_collection()    # collection 초기화
> ```

##### as_retriever

현재 db를 기반으로 VectorStoreRetriver 객체 생성. <br>

<br>

> ```python
> # similarity
> retriever = db.as_retriever()
> retriever.invoke('tf-idf에 대해서 알려줘')
> 
> # mmr (maximal marginal relavance)
> retriever = db.as_retriever(
>     searcy_type='mmr',
>     search_kwargs={
>         'fetch_k': 20,        # mmr에 전달할 문서 수
>         'k': 6,               # 반환 무서 수
>         'lambda_mult': 0.25,  # 다양성 조정 파라미터
>         # 'filter': {'source': './data/keywords.txt'}        # 메타데이터 기반 필터링
>     }
> )
> retriever.invoke('tf-idf에 대해서 알려줘')
>
> # score threshold
> retriever = db.as_retriever(
>     search_type='similarity_score_threshold',
>     search_kwargs={'score_threshold': 0.8},
> )
> retriever.invoke('tf-idf에 대해서 알려줘') 
> ```

In [249]:
nest_asyncio.apply()

In [256]:
# Practice
# 네이버 뉴스 경제 세션의 기사 크롤링
## 기사 url 찾기
response = requests.get('https://news.naver.com/section/101')
bs_response = BeautifulSoup(response.text, 'lxml')
urls = [
    item.get('href')
    for item
    in bs_response.select('div#newsct div.sa_text > a')
]

## 전체 기사 크롤링
news = naver_news_crawler(urls)

# splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)
documents = text_splitter.split_documents(
    news
)

# db (Chroma)
db_chroma = Chroma.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
    collection_name='news',
    persist_directory='./db'
)

# as_retriever
retriever_chroma = db_chroma.as_retriever()

### Retriever

저장된 벡터 데이터베이스에서 사용자의 질문과 관련된 문서를 검색하는 과정. <br> RAG 시스템의 전반적인 성능과 직결. <br>

<br>

<font style="font-size=20"> 특징 </font>

1\. 정확한 정보 제공: 사용자의 질문과 가장 관련성 높은 정보를 검색. <br>
시스템이 정확하고 유용한 답변을 생성할 수 있도록 함. <br>
이 과정이 효과적으로 이루어지지 않으면, 답변의 품질이 떨어질 수 있음. <br>

2\.응답 시간 단축: 효율적인 검색 알고리즘을 사용하여 데이터베이스에서 적절한 정보를 빠르게 검색함으로써, 전체적인 시스템 응답 시간 단축. <br>
사용자 경험 향상에 직접적인 영향. <br>

3\.최적화: 효과적인 검색 과정을 통해 필요한 정보만을 추출함. <br> 
-> 시스템 자원 사용 최적화, 불필요한 데이터 처리 감소.

<br>

<font style="font-size=20"> 동작 방식 </font>

1\. 질문의 벡터화: 사용자의 질문을 벡터 형태로 변환. <br>
임베딩 단계와 유사한 기술을 사용하여 진행되며, 변환된 질문 벡터는 후속 검색 작업의 기준점으로 사용. <br>

2\.벡터 유사성 비교: 저장된 문서 벡터와 질문 벡터 사이의 유사성을 계산. <br> cosine similarity,, Max Marginal Relevance(MMR) 등의 수학적 방법을 통하여 수행.

3\.상위 문서 선정: 계산된 유사성 점수를 기준으로 상위 N개의 가장 관련성 높은 문서 선정. <br>
이 문서들은 다음 단계에서 사용자의 질문에 대한 답변을 생성하는 데 사용. <br>

4\.문서 정보 반환: 선정된 문서들의 정보를 전달. <br>

<br>

<font style="font-size=20"> Sparse vs Dense </font>

Sparse Retriever와 Dense Retriever는 정보 검색 시스템에서 사용되는 두 가지 주요 방법. <br>
대규모 문서 집합에서 관련 문서를 검색할 때 사용. <br>

<br>

<font style="font-size=18"> Sparse Retriever </font>

문서와 질문을 이산적인 키워드 벡터로 변환하여 처리. <br>
TF-IDF나 BM25와 같은 정보 검색 기법 사용. <br>

- TF-IDF: 단어가 문서에 나타나는 빈도와 그 단어가 몇 개의 문서에서 나타나는지를 반영하여 단어의 중요도 계산. <br>
자주 나타나면서도 문서 집합 전체에서 드물게 나타나는 단어가 높은 가중치를 받음.
- BM25: TF-IDF를 개선한 모델. <br>
문서의 길이를 고려하여 검색 정확도를 향상. <br>
긴 문서와 짧은 문서 간의 가중치를 조정하여, 단어 빈도의 영향을 상대적으로 조절.

<br>

<font style="font-size=18"> Dense Retriever </font>

딥러닝 기법을 사용하여 문서와 쿼리를 연속적인 고차원 벡터로 인코딩. <br>
문서의 의미적 내용을 보다 풍부하게 표현 가능. <br>
키워드가 완벽히 일치하지 않더라도 의미적으로 관련된 문서를 검색 가능. <br>

벡터 공간에서의 거리를 사용하여 쿼리와 가장 관련성 높은 문서를 찾음. <br>
언어의 뉘앙스와 문맥을 이해하는 데 유리하며, 복잡한 쿼리에 대해 더 정확한 검색 결과를 제공 가능. <br>

||Sparse Retriever|Dense Retriever|
|-|----------------|---------------|
|표현 방식|이산적 키워드|연속 벡터|
|의미 처리 능력|문맥과 의미 파악 어려움, 키워드가 정확하게 일치해야 함|문맥과 의미 파악 가능, 키워드가 정확하게 일치하지 않아도 됨|
|적용 범위|간단, 명확한 키워드 검색| 복잡한 질문 등|

#### VectorStore Retriever

vector store를 사용하여 문서를 검색하는 retriever. <br>
similarity search나 MMR과 같은 기법을 통하여 vector store 내 텍스트 쿼리. <br>

<br>

> ```python
> # Retriever를 호출하여 주어진 쿼리에 대한 관련 문서를 반환
> retriever.invoke('tf-idf란 무엇인가?')
> 
> # Configurable
> retriever = db.as_retriever(
>     search_kwargs={'k': 1}
> ).configurable_fields(
>     search_type=ConfigurableField(
>         id='search_type',
>         name='Search Type',
>         description='검색 방법',
>     ),
>     search_kwargs=ConfigurableField(
>         id='search_kwargs',
>         name='Search Kwargs',
>         description='검색 kwargs',
>     ),
> )
> 
> # 상위 세 개 반환
> config = {'configurable': {'search_kwargs': {'k': 3}}}
> retriever.invoke('what embedding is?', config=config)
> 
> # 검색 설정 지정, score_threshold 0.7이상의 문서 반환
> config = {
>     'configurable': {
>         'search_type': 'similarity_score_threshold',
>         'search_kwargs': {
>             'score_threshold': 0.7,
>         },
>     }
> }
> retriever.invoke('what embedding is?', config=config)
> 
> # 검색 설정 지정, mmr 사용.
> config = {
>     'configurable': {
>         'search_type': 'mmr',
>         'search_kwargs': {
>             'k': 2,
>             'fetch_k': 10,
>             'lambda_mult': 0.6,
>         },
>     }
> }
> retriever.invoke('what embedding is?', config=config)
> ```

In [274]:
# db_chroma를 리트리버로 변환
retriever_chroma = db_chroma.as_retriever(
    # 검색할 결과의 개수를 1로 설정
    search_kwargs={'k': 1},    
).configurable_fields(
    # 검색 알고리즘을 선택할 수 있는 필드 설정
    search_type=ConfigurableField(
        id='search_type',  # 필드의 고유 ID
        name='Search Type',  # 사용자에게 보여질 필드 이름
        description='검색 알고리즘 선택'  # 필드에 대한 설명
    ),
    # 검색 알고리즘의 설정을 위한 필드 설정
    search_kwargs=ConfigurableField(
        id='search_kwargs',  # 필드의 고유 ID
        name='Search Kwargs',  # 사용자에게 보여질 필드 이름
        description='검색 알고리즘의 config'  # 필드에 대한 설명
    )
)

In [279]:
# retriever_chroma를 사용하여 '삼성전자'에 대한 검색 수행
results = retriever_chroma.invoke(
    '삼성전자',  # 검색할 쿼리
    config={  # 검색 설정을 포함하는 구성
        'configurable': {
            'search_type': 'mmr',  # 사용할 검색 알고리즘 타입 (MMR - Maximal Marginal Relevance)
            'search_kwargs': {  # 검색 알고리즘에 대한 추가 설정
                'k': 2,  # 반환할 결과의 수 (상위 2개 결과)
                'fetch_k': 10,  # 내부적으로 가져올 결과의 수 (10개)
            }
        }
    }
)

# 검색 결과 출력
results

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/018/0005868170'}, page_content='가족들을 비롯해 삼성 계열사 사장단 및 임직원 등 1000여 명이 참석했다. 공연장 로비에는 이 선대회장의 생전 사진과 삼성 경영과 관련해 당부했던 메시지가 전시됐다. ‘휴대폰 개발에 신경을 쓰십시오. 반드시 한 명당 한 대의 무선 단말기를 가지는 시대가 옵니다’(1995년), ‘미래 사회에는 손톱 크기의 반도체에 지구 상의 모든 정보를 담아 휴대가 가능해지고, 인간의 두뇌에 버금가는 인공지능(AI)이 개발될 것입니다’(2000년)등 이 선대회장의 주요 발언이 소개됐다.이 선대회장은 1987년 부친인 이병철 창업회장 별세 이후'),
 Document(metadata={'source': 'https://n.news.naver.com/mnews/article/422/0000689027'}, page_content='주가 반등세를 보이는 반면 삼성전자 주가가 연일 하락하면서 52주 신저가를 재차 경신했습니다. 외국인은 역대 최장인 33거래일 연속 삼성전자를 순매도하고 있는데 어떻습니까? 언제쯤 반등할 수 있을까요? 연합뉴스TV 기사문의 및 제보 : 카톡/라인 jebo23')]

#### ContextualCompressionRetriever

<img src="https://wikidocs.net/images/page/234097/01-Contextual-Compression.jpeg" width="400">

<br>

사용자의 질문에 대하여 관련성이 높은 정보가 많은 양의 무관한 텍스트를 포함한 문서에 묻혀 있을 수 있을 가능성이 높음. <br>
이러한 전체 문서를 전달하면 LLM 호출에 많은 비용이 들어가며, 낮은 품질의 응답으로 이어질 수 있음. <br>

ContextualCompressionRetriever 검색된 문서를 그대로 반환하는 대신, 주어진 질의의 맥락을 사용하여 문서를 압축함으로써 관련 정보만 반환되도록 함. <br>
- 압축: 개별 문서의 내용을 압축하는 것, 문서를 전체적으로 필터링하는 것

<br>

> ```python
> # EmbeddingsFilter: 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환하여 더 저렴하고 빠른 옵션 제공.
> embeddings_filter = EmbeddingsFilter(
>     embeddings=OpenAIEmbeddings(),
>     similarity_threshold=0.8,   # 유사도가 0.8 이상인 문서 필터
> )
> 
> compression_retriever = ContextualCompressionRetriever(
>     base_compressor=embeddings_filter,
>     base_retriever=retriever,
> )
> 
> compression_retriever.invoke(<question>)
> ```

In [285]:
# OpenAIEmbeddings를 사용하여 임베딩 필터를 생성
embeddings_filter = EmbeddingsFilter(
    embeddings=OpenAIEmbeddings(model='text-embedding-3-small'),  # 사용할 임베딩 모델 지정
    similarity_threshold=0.7,  # 유사도 임계값 설정 (0.7 이상인 경우만 필터링)
)

# ContextualCompressionRetriever를 생성하여 압축된 검색 결과를 반환
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,  # 임베딩 필터를 압축기로 사용
    base_retriever=retriever_chroma,     # 기본 리트리버로 retriever_chroma 사용
)

In [286]:
# 0.7 이상의 유사도를 가진 문서가 없음음
compression_retriever.invoke('삼성전자')

[]

#### EnsembleRetriever

EnsembleRetriever는 여러 검색기를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능. <br>
다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능을 달성 가능. <br>

<br>

<font style='font-size:20px'> 특징 </font>

1. 여러 검색기 통합: 다양한 유형의 검색기를 입력으로 받아 결과 결합.
2. 결과 재순위화: Reciprocal Rank Fusion 알고리즘을 사용하여 결과의 순위 조정.
3. 하이브리드 검색: 주로 sparse retriever와 dense retriever를 결합하여 사용.
- Sparse retriever: 키워드 기반 검색에 효과적
- Dense retriever: 의미적 유사성 기반 검색에 효과적

<br>

> ```python
> documents: list[str]
> 
> bm25_retriever = BM25Retriever.from_texts(
>     doc_list,
> )
> bm25_retriever.k = 1
> 
> faiss_vectorstore = FAISS.from_texts(
>     doc_list,
>     embedding=OpenAIEmbeddings(model_name='gpt-4o-mini'),
> )
> faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={'k': 1})
> 
> ensemble_retriever = EnsembleRetriever(
>     retrievers=[bm25_retriever, faiss_retriever],
>     weights=[0.7, 0.3],
> )
> 
> query = 'query'
> ensemble_result = ensemble_retriever.invoke(query)
> 
> # configurable
> ensemble_retriever = EnsembleRetriever(
>     retrievers=[bm25_retriever, faiss_retriever],
> ).configurable_fields(
>     weights=ConfigurableField(
>         id='ensemble_weights',
>         name='Ensemble Weights',
>         description='Ensemble Weights',
>     )
> )
> 
> config = {'configurable': {'ensemble_weights': [1, 0]}}
> ensemble_retriever.invoke('what embedding is', config=config)
> ```

In [288]:
# BM25Retriever를 사용하여 문서에서 검색을 수행하는 리트리버 생성
retriever_b25 = BM25Retriever.from_documents(news)
retriever_b25.k = 1  # 반환할 결과의 개수를 1로 설정

# Chroma를 사용하여 문서로부터 임베딩 데이터베이스 생성
db_chroma_temp = Chroma.from_documents(
    news,  # 사용될 문서
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),  # 사용할 임베딩 모델 지정
)

# Chroma 데이터베이스를 리트리버로 변환하고 검색할 결과의 개수를 1로 설정
retriever_temp_chroma = db_chroma_temp.as_retriever(search_kwargs={'k': 1})

# 앙상블 리트리버 생성: 두 개의 리트리버를 결합하여 결과를 통합
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever_b25, retriever_temp_chroma],  # 사용할 리트리버 목록
    weights=[0.5, 0.5],  # 각 리트리버에 대한 가중치 (합이 1이 되도록 설정)
).configurable_fields(
    # 앙상블 리트리버의 가중치를 설정할 수 있는 필드 추가
    weights=ConfigurableField(
        id='ensemble_weights',  # 필드의 고유 ID
        name='Ensemble Weights',  # 사용자에게 보여질 필드 이름
        description='앙상블된 두 retriever의 비율 (비율의 합은 1)'  # 필드에 대한 설명
    )
)

In [290]:
# ensemble_retriever를 사용하여 '삼성전자'에 대한 검색 수행
results = ensemble_retriever.invoke(
    '삼성전자',  # 검색할 쿼리
    config={  # 검색 설정을 포함하는 구성
        'configurable': {
            'ensemble_weights': [0.2, 0.8]  # 각 리트리버에 대한 가중치를 설정 (비율의 합은 1)
        }
    }
)

# 검색 결과 출력
results

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/008/0005105524'}, page_content='\nLG전자, 아쉬운 3분기 실적…장중 5%대 급락\n\n\n\n입력2024.10.25. 오후 1:18\n\n\n수정2024.10.25. 오후 1:19\n\n기사원문\n \n\n\n\n\n김진석 기자\n\nTALK\n\n\n\n\n\n\n\n\n\n김진석 기자\n\nTALK\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n[특징주]\n\n\n\n /사진=임종철 기자LG전자가 장 중 5%대 급락 중이다. 25일 오후 1시 10분 기준 코스피 시장에서 LG전자는 전날보다 4900원(5.04%) 하락한 9만2300원에 거래 중이다. 3분기 시장 기대치에 못 미친 실적을 내면서 투자심리가 악

#### LongContextReorder

paper: https://arxiv.org/pdf/2307.03172

모델이 긴 컨텍스트 중간에 있는 관련 정보에 접근해야 할 때, 제공된 문서를 무시하는 경향이 있음. <br>
-> 검색 후 문서의 순서를 재배열하여 성능 저하 방지. 

<br>

> ```python
> embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
> texts = [
>     "영화는 다양한 이야기를 시각적으로 표현하는 매체입니다.",
>     "좋은 영화는 관객의 감정을 깊이 있게 자극합니다.",
>     "영화의 배경 음악은 이야기의 분위기를 한층 더 풍부하게 만듭니다.",
>     "고전 영화는 오늘날에도 여전히 많은 사랑을 받고 있습니다.",
>     "상상력이 풍부한 영화는 관객을 새로운 세계로 안내합니다.",
>     "여행은 새로운 경험과 추억을 만들어 줍니다.",
>     "커피 한 잔과 좋은 책은 완벽한 조합입니다.",
>     "친구와의 대화는 언제나 즐거운 시간이 됩니다.",
>     "가을의 바람은 특별한 감성을 불러일으킵니다.",
>     "정원에서 꽃을 가꾸는 것은 마음을 편안하게 합니다."
> ]
> 
> retriever = Chroma.from_texts(
>     texts,
>     embedding=embeddings
>     ).as_retriever(search_kwargs={'k': 10}
> )
> 
> reordering = LongContextReorder()
> documents_reordered = reordering.transform_documents(documents)
> 
> ###################################
> # 사용 예제
> 
> def reorder_documents(documents):
>     reordering = LongContextReorder()
>     reordered_documents = reordering.transform_documents(documents)
>     documents_joined = '\n'.join([document.page_content for document in docs])
> 
>     return documents_joined
> 
> template = '''
> 주어진 context를 활용하라:
> {context}
> 
> 다음 질문에 답하라:
> {question}
> 
> 주어지는 언어로 답변하라: {language}
> '''
> 
> prompt = ChatPromptTemplate.from_template(template)
> model = ChatOpenAI(model='gpt-4o-mini')
> parser = StrOutputParser()
> 
> chain = (
>     {
>         'context': itemgetter('question')
>         | retriever
>         | RunnableLambda(reorder_documents),
>         'question': itemgetter('question'),
>         'language': itemgetter('language'),
>     }
>     | prompt
>     | model
>     | parser
> )
> 
> answer = chain.invoke(
>     {'question': 'ChatGPT에 대해 무엇을 말해줄 수 있나요?', 'language': 'KOREAN'}
> )
> ```

In [291]:
texts = [
    "영화는 다양한 이야기를 시각적으로 표현하는 매체입니다.",
    "좋은 영화는 관객의 감정을 깊이 있게 자극합니다.",
    "영화의 배경 음악은 이야기의 분위기를 한층 더 풍부하게 만듭니다.",
    "고전 영화는 오늘날에도 여전히 많은 사랑을 받고 있습니다.",
    "상상력이 풍부한 영화는 관객을 새로운 세계로 안내합니다.",
    "여행은 새로운 경험과 추억을 만들어 줍니다.",
    "커피 한 잔과 좋은 책은 완벽한 조합입니다.",
    "친구와의 대화는 언제나 즐거운 시간이 됩니다.",
    "가을의 바람은 특별한 감성을 불러일으킵니다.",
    "정원에서 꽃을 가꾸는 것은 마음을 편안하게 합니다."
]

In [ ]:
# Chroma를 사용하여 텍스트 데이터로부터 임베딩 데이터베이스 생성
retriever_movie = Chroma.from_texts(
    texts,  # 사용할 텍스트 데이터 (문서 목록)
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),  # 사용할 임베딩 모델 지정
).as_retriever(search_kwargs={'k': 10})  # 검색할 결과의 개수를 10으로 설정

In [293]:
# LongContextReorder 객체 생성: 긴 문맥을 재정렬하는 데 사용
context_reorder = LongContextReorder()

# 원본 텍스트 문서들을 재정렬
texts_reordered = context_reorder.transform_documents(texts)

# 재정렬된 텍스트 출력
texts_reordered

['좋은 영화는 관객의 감정을 깊이 있게 자극합니다.',
 '고전 영화는 오늘날에도 여전히 많은 사랑을 받고 있습니다.',
 '여행은 새로운 경험과 추억을 만들어 줍니다.',
 '친구와의 대화는 언제나 즐거운 시간이 됩니다.',
 '정원에서 꽃을 가꾸는 것은 마음을 편안하게 합니다.',
 '가을의 바람은 특별한 감성을 불러일으킵니다.',
 '커피 한 잔과 좋은 책은 완벽한 조합입니다.',
 '상상력이 풍부한 영화는 관객을 새로운 세계로 안내합니다.',
 '영화의 배경 음악은 이야기의 분위기를 한층 더 풍부하게 만듭니다.',
 '영화는 다양한 이야기를 시각적으로 표현하는 매체입니다.']

In [294]:
def reorder_documents(documents):
    # LongContextReorder 객체 생성: 긴 문맥을 재정렬하는 기능
    context_reorder = LongContextReorder()
    
    # 입력된 문서들을 재정렬
    documents_reordered = context_reorder.transform_documents(documents)
    
    # 재정렬된 문서의 내용을 줄바꿈으로 구분하여 하나의 문자열로 결합
    documents_joined = '\n'.join([document.page_content for document in documents_reordered])

    return documents_joined  # 재정렬된 문서의 내용을 반환

In [ ]:
# 프롬프트 템플릿 정의: reference, question, language를 포함
template = '''
    주어진 reference를 최대한 활용하라:
    {reference}

    다음 질문에 답하라:
    {question}

주어지는 언어로 답하라: {language}
'''

# 템플릿으로부터 프롬프트 생성
prompt = PromptTemplate.from_template(template)

# OpenAI의 Chat 모델 초기화 (gpt-4o-mini 모델 사용)
model = ChatOpenAI(model_name='gpt-4o-mini')

# 출력 파서를 초기화 (문자열 출력을 처리)
parser = StrOutputParser()

# 체인 구성: 데이터 흐름을 정의
chain = (
    {
        # 'reference' 키에 대해 여러 처리를 정의
        'reference': itemgetter('question')  # 질문에서 reference를 추출
        | retriever_chroma  # Chroma 리트리버로부터 데이터를 검색
        | RunnableLambda(reorder_documents),  # 문서를 재정렬
        'question': itemgetter('question'),  # 질문을 그대로 가져오기
        'language': itemgetter('language'),  # 언어 정보를 가져오기
    }
    | prompt  # 프롬프트 템플릿에 데이터 결합
    | model  # 모델에 프롬프트 전달하여 응답 생성
    | parser  # 모델의 응답을 파싱
)

In [304]:
# 체인을 사용하여 질문을 실행하고 응답을 받음
response = chain.invoke({
    'question': '삼성전자의 최근 이슈에 대해서 알려줄래?',  # 질문: 삼성전자의 최근 이슈
    'language': '한국어'  # 언어: 한국어로 응답 요청
})

# 응답 출력
print(response)

"삼성전자는 최근 다양한 이슈에 직면해 있습니다. 이재용 삼성전자 회장이 취임 2주년을 맞이한 가운데, 고 이건희 선대 회장의 4주기를 맞이하여 삼성에서 어떤 공개 메시지가 나올지 관심이 집중되고 있습니다. 또한, 삼성전자는 '위기론'에 대한 우려가 커지고 있으며, 이는 기업의 실적 부진과 관련이 깊습니다. \n\n특히, SK하이닉스가 어닝서프라이즈를 기록하며 주가가 반등하고 있는 반면, 삼성전자는 지속적인 주가 하락으로 52주 신저가를 기록하고 있어 시장에서의 우려가 커지고 있습니다. 이러한 상황 속에서 삼성전자의 향후 대응과 전략이 주목받고 있으며, 기업 실적과 글로벌 경제 상황에 대한 경계감이 고조되고 있습니다."

In [317]:
# nlp_keywords -> retriever 구축
# 텍스트 파일 로더 생성: 지정된 경로에서 텍스트 파일을 로드
loader = TextLoader('./data/nlp_keywords.txt')

# 텍스트 분할기 생성: 각 청크의 크기와 중첩 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,  # 각 청크의 최대 크기 (250자)
    chunk_overlap=50,  # 각 청크 간의 중첩 (50자)
)

# 텍스트를 로드하고 분할하여 문서 리스트 생성
documents = loader.load_and_split(text_splitter)

# Chroma 데이터베이스 생성: 문서들을 임베딩하여 저장
db_nlp = Chroma.from_documents(
    documents,  # 사용될 문서 리스트
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),  # 사용할 임베딩 모델 지정
)

# Chroma 데이터베이스를 리트리버로 변환하고 검색할 결과의 개수를 10으로 설정
retriever_nlp = db_nlp.as_retriever(search_kwargs={'k': 10})

# 문서 재정렬 함수 정의
def reorder_documents(documents):
    # LongContextReorder 객체 생성: 긴 문맥을 재정렬하는 기능
    context_reorder = LongContextReorder()
    
    # 입력된 문서들을 재정렬
    documents_reordered = context_reorder.transform_documents(documents)
    
    # 재정렬된 문서의 내용을 줄바꿈으로 구분하여 하나의 문자열로 결합
    documents_joined = '\n'.join([document.page_content for document in documents_reordered])

    return documents_joined  # 재정렬된 문서의 내용을 반환

# 프롬프트 템플릿 정의: reference, question, language를 포함
template = '''
주어진 reference를 최대한 활용하라:
{reference}

다음 질문에 답하라:
{question}

주어지는 언어로 답하라: {language}
'''

## question과 연관된 reference를 retriever를 통해 획득
## 획득한 reference를 prompt에 주입
## prompt를 llm에 입력하여 결과 return
# 프롬프트 템플릿을 사용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# OpenAI의 Chat 모델 초기화 (gpt-4o-mini 모델 사용)
model = ChatOpenAI(model_name='gpt-4o-mini')

# 문자열 출력을 처리하기 위한 출력 파서 초기화
parser = StrOutputParser()

# 데이터 흐름을 정의하는 체인 구성
chain = (
    {
        # 'reference' 키에 대한 처리 흐름 정의
        'reference': itemgetter('question')  # 질문에서 reference를 추출
        | retriever_nlp  # NLP 리트리버를 사용하여 데이터를 검색
        | RunnableLambda(reorder_documents),  # 검색된 문서를 재정렬
        'question': itemgetter('question'),  # 질문을 그대로 가져오기
        'language': itemgetter('language'),  # 언어 정보를 가져오기
    }
    | prompt  # 프롬프트 템플릿에 데이터 결합
    | model  # 모델에 프롬프트를 전달하여 응답 생성
    | parser  # 모델의 응답을 파싱
)

In [318]:
# 체인을 사용하여 질문을 실행하고 응답을 받음
response = chain.invoke({
    'question': 'huggingface에서 deep learning을 어떻게 사용할 수 있어?',
    'language': '한국어'
})

# 응답 출력
print(response)

'HuggingFace에서는 딥러닝을 다양한 방식으로 사용할 수 있습니다. 주로 자연어 처리(NLP) 작업을 위한 사전 훈련된 모델과 도구를 제공하여, 사용자가 쉽게 딥러닝 모델을 활용할 수 있도록 돕습니다. \n\n예를 들어, HuggingFace의 Transformers 라이브러리를 사용하면 텍스트 분류, 감정 분석, 번역, 텍스트 생성 등 여러 NLP 작업을 간편하게 수행할 수 있습니다. 이러한 모델들은 대규모 데이터셋으로 사전 훈련되어 있어, 특정 작업에 맞게 추가 훈련(fine-tuning)하여 높은 성능을 발휘할 수 있습니다.\n\n또한, HuggingFace는 사용자 친화적인 API를 제공하여, 복잡한 딥러닝 모델을 손쉽게 사용할 수 있도록 해줍니다. 이를 통해 데이터 분석가나 개발자는 딥러닝의 복잡성을 최소화하면서도 효과적인 솔루션을 구축할 수 있습니다.'

In [321]:
# NLP 리트리버를 사용하여 질문에 대한 응답을 검색
response = retriever_nlp.invoke('huggingface에서 deep learning을 어떻게 사용할 수 있어?')  # 질문: Hugging Face에서 딥 러닝 사용 방법

# 응답 출력
response

[Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='HuggingFace\n\n정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nTransformer'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='Deep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nDataFrame'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='판다스 (Pandas)\n\n정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조작 도구를 제공하는 라이브러리입니다.\n예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며, 다양한 분석을 수행할 수 있습니다.\n연관키워드: 데이터 분석, 파이썬, 데이터 처리\n\nDeep Learning'),
 Document(metadata={'source': './data/nlp_keywords.txt'}, page_content='LLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nHuggingFace'),
 Document(metad

#### ParentDocumentRetriever

문서 검색 과정에서 문서를 적절한 크기의 chunk로 나누는 것은 다음의 상충되는 두 가지 중요한 요소를 고려해야 함.

1. 작은 문서를 원하는 경우: 문서의 임베딩이 그 의미를 가장 정확하게 반영. <br>
문서가 너무 길면 임베딩이 의미를 잃어버릴 수 있음. <br>
2. 각 청크의 맥락이 유지되도록 충분히 긴 문서를 원하는 경우.

위의 두 경우에서 균형을 맞추기 위해 ParentDocumentRetriever라는 도구가 사용. <br> 
문서를 작은 chunk으로 나누고, 이 chunk를 관리. <br>
검색을 진행할 때는, 먼저 이 작은 chunk들을 찾아낸 다음, 이 chunk들이 속한 원본 문서의 ID를 통해 전체적인 맥락을 파악 가능. <br>

parent document: 원본 문서. 전체 문서일 수도 있고, 비교적 큰 다른 chunk일 수도 있음. <br>
이 방식을 통해 문서의 의미를 정확하게 파악하면서도, 전체적인 맥락을 유지할 수 있음 <br>

<br>

> ```python
> retriever = ParentDocumentRetriever(
>     vectorstore=vectorstore,
>     docstore=store,
>     child_splitter=child_splitter,
> )
> 
> parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
> child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
> vectorstore = Chroma(
>     collection_name='split_parents',
>     embedding_function=OpenAIEmbeddings(),
> )
> store = InMemoryStore()
> 
> retriever = ParentDocumentRetriever(
>     vectorstore=vectorstore,
>     docstore=store,
>     child_splitter=child_splitter,
>     parent_splitter=parent_splitter,
> )
> ```

In [3]:
# 텍스트 파일 로더 생성: 지정된 경로에서 텍스트 파일을 로드
loader = TextLoader('./data/finance_keywords.txt')

# 텍스트 분할기 생성: 각 청크의 크기 설정
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200  # 각 청크의 최대 크기 (200자)
)

# Chroma 데이터베이스 생성: 금융 관련 문서 저장
db_finance = Chroma(
    collection_name='full_documents',  # 컬렉션 이름 설정
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-small')  # 사용할 임베딩 모델 지정
)

# 인메모리 스토어 생성: 문서 저장을 위한 임시 스토리지
store = InMemoryStore()

# 부모 문서 리트리버 생성: Chroma 데이터베이스와 스토어를 연결
retriever = ParentDocumentRetriever(
    vectorstore=db_finance,  # 벡터 스토어 설정
    docstore=store,  # 문서 스토어 설정
    child_splitter=child_splitter,  # 자식 문서 분할기 설정
)

# 로드한 문서를 리트리버에 추가
retriever.add_documents(loader.load())

In [8]:
# '비트코인'과 관련된 문서를 Chroma 데이터베이스에서 검색하고, 첫 번째 결과의 내용을 출력
result = db_finance.similarity_search('비트코인')  # '비트코인'에 대한 유사도 검색 수행
print(result[0].page_content)  # 검색 결과 중 첫 번째 문서의 내용을 출력

환율 (Exchange Rate)

정의: 환율은 두 통화 간의 교환 비율을 의미합니다.
예시: 1달러가 1,200원일 경우, 달러와 원화의 환율이 1,200입니다.
연관키워드: 외환, 금융, 경제

신용 (Credit)


In [9]:
# 부모 문서와 자식 문서를 위한 텍스트 분할기 생성
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)  # 부모 문서의 청크 크기를 1000자로 설정
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)  # 자식 문서의 청크 크기를 200자로 설정

# Chroma 데이터베이스 생성: 부모 문서를 저장하기 위한 데이터베이스
db_finance_with_parent = Chroma(
    collection_name='parent_documents',  # 데이터베이스의 컬렉션 이름을 'parent_documents'로 설정
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-small')  # 사용할 임베딩 모델 지정
)

# 인메모리 스토어 생성: 문서를 저장할 인메모리 스토어
store = InMemoryStore()

# 부모 문서 리트리버 생성: 부모 문서와 자식 문서를 검색할 리트리버
retriever_parent = ParentDocumentRetriever(
    vectorstore=db_finance_with_parent,  # 벡터 스토어로 부모 문서를 저장한 Chroma 데이터베이스 지정
    docstore=store,  # 문서 저장소로 인메모리 스토어 지정
    child_splitter=child_splitter,  # 자식 문서 분할기를 지정
    parent_splitter=parent_splitter,  # 부모 문서 분할기를 지정
)

# 로드한 문서를 리트리버에 추가
retriever_parent.add_documents(loader.load())  # 로더를 통해 로드한 문서들을 추가

In [10]:
# 인메모리 스토어에 저장된 키의 개수를 세어 출력
num_keys = len(list(store.yield_keys()))  # yield_keys()를 호출하여 저장된 키 목록을 생성하고, 그 길이를 계산
print(num_keys)  # 저장된 키의 개수를 출력

3

In [12]:
# '채권'과 관련된 문서를 Chroma 데이터베이스에서 검색하고, 첫 번째 결과의 내용을 출력
result = db_finance_with_parent.similarity_search('채권')  # '채권'에 대한 유사도 검색 수행
print(result[0].page_content)  # 검색 결과 중 첫 번째 문서의 내용을 출력

채권 (Bond)

정의: 채권은 발행자가 일정 기간 후에 원금과 이자를 지급하겠다고 약속하는 부채 증서입니다.
예시: 정부가 인프라 프로젝트를 위해 채권을 발행할 수 있습니다.
연관키워드: 고정 수익, 투자, 자금 조달

유동성 (Liquidity)


In [16]:
# 부모 문서 리트리버를 사용하여 '채권'에 대한 응답을 검색하고, 첫 번째 결과의 내용을 출력
response = retriever_parent.invoke('채권')  # '채권'에 대한 질문으로 리트리버에 요청
print(response[0].page_content)  # 검색 결과 중 첫 번째 문서의 내용을 출력

자본 (Capital)

정의: 자본은 기업이 사업을 운영하기 위해 사용하는 자원의 총액을 의미합니다.
예시: 기업은 자본을 통해 새로운 프로젝트에 투자하거나 운영 비용을 충당할 수 있습니다.
연관키워드: 재무, 투자, 자산

주식 (Stock)

정의: 주식은 기업의 소유권을 나타내는 증권으로, 주식을 소유한 사람은 해당 기업의 일부를 소유하게 됩니다.
예시: 주식을 구매하면 기업의 이익 배당금을 받을 수 있습니다.
연관키워드: 증권, 투자, 자본 시장

채권 (Bond)

정의: 채권은 발행자가 일정 기간 후에 원금과 이자를 지급하겠다고 약속하는 부채 증서입니다.
예시: 정부가 인프라 프로젝트를 위해 채권을 발행할 수 있습니다.
연관키워드: 고정 수익, 투자, 자금 조달

유동성 (Liquidity)

정의: 유동성은 자산을 현금으로 전환하는 용이성을 의미합니다.
예시: 주식은 상대적으로 높은 유동성을 가지지만, 부동산은 낮은 유동성을 가질 수 있습니다.
연관키워드: 자산, 금융, 시장

금리 (Interest Rate)

정의: 금리는 대출이나 투자에 대한 이자의 비율을 나타내며, 경제에 큰 영향을 미칩니다.
예시: 중앙은행이 금리를 인상하면 대출이 줄어들 수 있습니다.
연관키워드: 금융 정책, 대출, 투자

포트폴리오 (Portfolio)

정의: 포트폴리오는 투자자가 보유한 다양한 자산의 집합을 의미합니다.
예시: 주식, 채권, 부동산 등으로 구성된 포트폴리오가 있을 수 있습니다.
연관키워드: 투자, 위험 관리, 자산 배분

리스크 (Risk)

정의: 리스크는 투자에서 예상치 못한 손실이 발생할 가능성을 의미합니다.
예시: 주식 시장의 변동성은 높은 리스크를 동반할 수 있습니다.
연관키워드: 투자, 손실, 위험 관리

배당금 (Dividend)

정의: 배당금은 기업이 이익의 일부를 주주에게 분배하는 금액을 의미합니다.
예시: 정기적으로 배당금을 지급하는 안정적인 기업들이 많습니다.
연관키워드: 주식, 투자, 수익


### Practice

1\. 조선, 중앙, 동아일보 전체페이지 크롤링 <br>
2\. 데이터 db에 적재 (chroma) <br>
3\. retriever 생성 <br>
4\. chain 생성 <br>
5\. 조회 <br>

In [3]:
def get_page_urls(url):
    # 주어진 URL에 GET 요청을 보냅니다.
    response = requests.get(url)
    
    # 응답의 HTML 내용을 lxml 파서를 사용하여 BeautifulSoup 객체로 변환합니다.
    bs_response = BeautifulSoup(response.text, 'lxml')
    
    # 'a.nclicks(cnt_flashart)' 선택자로 링크를 찾아 href 속성을 추출합니다.
    urls = [
        item.get('href')
        for item in bs_response.select('a.nclicks\(cnt_flashart\)')
    ]

    # 추출한 URL 리스트를 반환합니다.
    return urls

In [4]:
# 빈 리스트를 초기화하여 URL을 저장할 준비를 합니다.
urls = []

# tqdm을 사용하여 진행 상황을 시각적으로 보여줍니다.
for oid in tqdm(('023', '025', '020')):
    # 각 oid에 대해 1페이지부터 19페이지까지 반복합니다.
    for page in range(1, 20):
        # 각 페이지의 URL을 생성합니다.
        url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date=20241025&page={page}'
        
        # 생성된 URL에서 페이지 URL을 추출하고 urls 리스트에 추가합니다.
        urls.extend(get_page_urls(url))

# 중복된 URL을 제거하여 고유한 URL 리스트로 만듭니다.
urls = list(set(urls))

100%|██████████| 3/3 [00:13<00:00,  4.50s/it]


In [5]:
nest_asyncio.apply()
news = naver_news_crawler(urls)

Fetching pages:   0%|          | 0/641 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 641/641 [00:28<00:00, 22.40it/s]


In [7]:
# 부모 문서와 자식 문서를 위한 텍스트 분할기 생성
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)  # 부모 문서의 청크 크기를 1000자로 설정
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)  # 자식 문서의 청크 크기를 200자로 설정

# Chroma 데이터베이스 생성: 부모 문서를 저장하기 위한 데이터베이스
db_finance_with_parent = FAISS(
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-small'),  # 사용할 임베딩 모델 지정
    index=faiss.IndexFlatL2(1536),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# 인메모리 스토어 생성: 문서를 저장할 인메모리 스토어
store = InMemoryStore()

# 부모 문서 리트리버 생성: 부모 문서와 자식 문서를 검색할 리트리버
retriever_parent = ParentDocumentRetriever(
    vectorstore=db_finance_with_parent,  # 벡터 스토어로 부모 문서를 저장한 Chroma 데이터베이스 지정
    docstore=store,  # 문서 저장소로 인메모리 스토어 지정
    child_splitter=child_splitter,  # 자식 문서 분할기를 지정
    parent_splitter=parent_splitter,  # 부모 문서 분할기를 지정
)

# 로드한 문서를 리트리버에 추가
retriever_parent.add_documents(news)  # 로더를 통해 로드한 문서들을 추가

In [10]:
# 문서 재정렬 함수 정의
def reorder_documents(documents):
    # LongContextReorder 객체 생성: 긴 문맥을 재정렬하는 기능
    context_reorder = LongContextReorder()
    
    # 입력된 문서들을 재정렬
    documents_reordered = context_reorder.transform_documents(documents)
    
    # 재정렬된 문서의 내용을 줄바꿈으로 구분하여 하나의 문자열로 결합
    documents_joined = '\n'.join([document.page_content for document in documents_reordered])

    return documents_joined  # 재정렬된 문서의 내용을 반환

# 프롬프트 템플릿 정의: reference, question, language를 포함
template = '''
주어진 reference를 최대한 활용하여 아래의 질문에 답하라:
{reference}

다음 질문에 답하라. reference에서 답을 찾을 수 없으면 찾을 수 없다고 답하라:
{question}

주어지는 언어로 답하라:
{language}
'''

## question과 연관된 reference를 retriever를 통해 획득
## 획득한 reference를 prompt에 주입
## prompt를 llm에 입력하여 결과 return
# 프롬프트 템플릿을 사용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# OpenAI의 Chat 모델 초기화 (gpt-4o-mini 모델 사용)
model = ChatOpenAI(model_name='gpt-4o-mini')

# 문자열 출력을 처리하기 위한 출력 파서 초기화
parser = StrOutputParser()

# 데이터 흐름을 정의하는 체인 구성
chain = (
    {
        # 'reference' 키에 대한 처리 흐름 정의
        'reference': itemgetter('question')  # 질문에서 reference를 추출
        | retriever_parent  # NLP 리트리버를 사용하여 데이터를 검색
        | RunnableLambda(reorder_documents),  # 검색된 문서를 재정렬
        'question': itemgetter('question'),  # 질문을 그대로 가져오기
        'language': itemgetter('language'),  # 언어 정보를 가져오기
    }
    | prompt  # 프롬프트 템플릿에 데이터 결합
    | model  # 모델에 프롬프트를 전달하여 응답 생성
    | parser  # 모델의 응답을 파싱
)

In [11]:
chain.invoke({'question': '삼성전자 근황이 어떻게 돼?', 'language': 'korean'})

'삼성전자는 최근 3분기 실적이 부진한 상황으로 보입니다. 증권가는 삼성전자의 반도체 부문 3분기 영업이익을 5조원 안팎으로 추산하고 있으며, SK하이닉스가 분기 영업이익 기준으로 처음으로 삼성 반도체를 제친 것으로 나타났습니다. 삼성전자의 반도체 부문장이 "걱정을 끼쳐 송구하다"는 사과문을 발표한 것으로 보아, 현재 삼성전자는 어려운 상황에 처해 있는 것으로 판단됩니다.'